# Pseudo Code for ensemble deep learning method based on optimization of skill scoresn and value-weighted skill scores
## Sabrina Guastavino, guastavino@dima.unige.it

In [1]:
import os

import csv
import numpy as np
import random
import pandas
import pickle
import glob
import os.path
import sys

from keras.utils import to_categorical
import matplotlib
from keras.callbacks import ModelCheckpoint, CSVLogger

import matplotlib.pyplot as plt
import matplotlib as mpl

import time
import os.path
import numpy
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

from keras import backend as K
import tensorflow as tf

In [2]:
import utilities_scores as utilities

In [3]:
"""

A collection of models we'll use to attempt to classify videos.

"""
import keras
import numpy
from keras.layers import Dense, Flatten, Dropout
from keras.layers.recurrent import LSTM

from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.wrappers import TimeDistributed

from keras.layers import (Conv2D,MaxPooling2D)


from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.models import Input

from tensorflow.keras import regularizers

from keras.layers import Activation
from sklearn.metrics import confusion_matrix


from keras import backend as K
import tensorflow as tf

from tensorflow.python.util.tf_export import tf_export
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import array_ops


from collections import deque
import sys



### Definition of the loss function 

In [4]:
# Definition of weighted categorical crossentropy
def weightedCatCELoss():
    def weightLoss_(y_true,y_pred):

        classes = K.argmax(y_true)
        classCount = K.sum(y_true,axis=0)

        loss = K.categorical_crossentropy(y_true, y_pred, from_logits=False)
        return loss / K.gather(classCount, classes)

    return weightLoss_

### Set the input shape variables

In [5]:
#set the variables of input shape

#number of images in time series
seq_length = 10
#size of images
image_shape = (128,256)
number_channels = 3


### Definition of the deep neural network model:
#### Long-term recurrent neural network (LRCN)

In [6]:
#LRCN MODEL

# the input shape is composed by (number of images in time series, image height, image width, number of channels)

#in the case no multi channel data are used set number_of_channels equal to 1

input_shape = (seq_length, image_shape[0], image_shape[1], number_channels)

initialiser = 'glorot_uniform'

reg_lambda  = 0.01

model = Sequential()


# first block
model.add(TimeDistributed(Conv2D(8, (5, 5), strides=(2, 2), padding='same', 
                                               kernel_initializer=initialiser) , #, 
                                  input_shape=input_shape)) 

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((4, 4), strides=(2, 2))))


# second block
model.add(TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2), padding='same',
                                               kernel_initializer=initialiser), 
                                  input_shape=input_shape)) 

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((4, 4), strides=(2, 2))))
  
# third block
model.add(TimeDistributed(Conv2D(32, (3, 3), strides=(2, 2), padding='same', 
                                               kernel_initializer=initialiser), 
                                  input_shape=input_shape)) 
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((4, 4), strides=(2, 2))))



# LSTM 
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, return_sequences=False, dropout=0.5)) 

model.add(Dense(2, activation='softmax'))



loss = weightedCatCELoss()
metrics=['accuracy']
optimizer = Adam(lr=1e-3, decay=1e-6)

model.compile(loss=loss, optimizer=optimizer,
                               metrics=metrics)

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 64, 128, 8)    608       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 64, 128, 8)    32        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 64, 128, 8)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 31, 63, 8)     0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 16, 32, 16)    1168      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 16, 32, 16)    64        
_________________________________________________________________
time_distributed_6 (TimeDist (None, 10, 16, 32, 16)    0

### Definition of the training function 

In [7]:


# DEFINE TRAINING

def train(model,batch_size=72, nb_epoch=100, 
          file_X_train='', file_y_train='',
         file_X_val='', file_y_val=''):
    
    # function which returns the trained deep neural network given in input
    # - model = the deep neural network model
    # - batch_size = the size of the batch size
    # - nb_epoch = number of epochs
    # - file_X_train = the file in which the training X matrix is saved
    # - file_y_train = the file in which the training y label vector is saved
    # - file_X_val = the file in which the validation X matrix is saved
    # - file_y_val = the file in which the validation y label vector is saved
    
    # tarining and validation data have to satisfy the following conditions:
    # X_train must be of size (n_train, seq_length, image_shape[0], image_shape[1], number_channels)
    # X_val must be of size (n_val, seq_length, image_shape[0], image_shape[1], number_channels)
    # y_train is a binary vector of size n_train 
    # y_val is a binary vector of size n_val

    # save the models for each epoch in data/checkpoints
    checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints',  'lrcn_train_3levels_weighted_cat_cross_'+str(nb_epoch)+'epoch'+ 
            '.{epoch:03d}-{val_loss:.3f}.hdf5'), 
        verbose=1,
        save_best_only=False) #

    # Helper: Save results.
    timestamp = time.time()
    csv_logger = CSVLogger(os.path.join('data', 'logs', 'lrcn-' + 'training-' + \
        str(timestamp) + '.log'))

    
    # Load training and validation data
    X_train = numpy.load(file_X_train)
    yy_train =  numpy.load(file_y_train)
    y_train = to_categorical(yy_train) #<-- with categorical crossentropy
    X_val = numpy.load(file_X_val)
    yy_val =  numpy.load(file_y_val)
    y_val = to_categorical(yy_val)  #<-- with categorical crossentropy
    model.fit(
            X_train,
            y_train,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            verbose=1,
            callbacks=[csv_logger, checkpointer], 
            epochs=nb_epoch)
    
    # save the history
    history_file = open('lrcn_history_train_3levels_batch_'+str(batch_size)+'_epoch_'+str(nb_epoch)+
                  '_weighted_cat_cross.pkl', "wb")
    pickle.dump(model.history.history, history_file)
    history_file.close()

    
    #plot the loss function on training and validation set over epochs
    mpl.style.use('default')
    fig = plt.figure()
    ax = fig.gca()
    plot_train = ax.plot(np.arange(1,len(model.history.history['loss'])+1),model.history.history['loss'],label='train loss',color='red', linewidth=1.9)
    plot_val = ax.plot(np.arange(1,len(model.history.history['val_loss'])+1),model.history.history['val_loss'],label='val loss',color='blue',linewidth=1.9)
    ax.set_xlabel('Epoch', fontsize=14)
    #ax.set_xlim(1,len(model_history['val_loss']))
    ax.legend(loc="upper right", fontsize=14)
    plt.savefig("plot_loss_lrcn_train_3levels_batch_"+str(batch_size)+"_epoch_"+str(nb_epoch)+
            "_weighted_cat_cross.png")
    
   
    return model


## Ensemble method

### (1) Train the deep neural network

In [ ]:
# TRAIN the deep neural network

seq_length = 10
image_shape = (128,256)
batch_size = 72 
nb_epoch = 2 


date_start_train ='20180709140000'#'20180902000000'
date_end_train ='20190718230000'

date_start_val = '20190719170000'
date_end_val='20191003000000'

# Define the filenames of the training set: the training matrix X_train and the label vector y_train
file_X_train = ''
file_y_train = ''
# Define the filenames of the validation set: the validation matrix X_val and the label vector y_val
file_X_val = ''
file_y_val = ''


model_history=train(model,batch_size=batch_size, nb_epoch=nb_epoch, 
                    file_X_train=file_X_train, file_y_train=file_y_train,
                    file_X_val=file_X_val, file_y_val=file_y_val)


### (2) Computation of optimal classification thresholds

In [ ]:
# For each epoch compute the optimal classification threshold by optimizing a desired skill 
# score on the training set and compute the desired skill score on the validation set


import gc


folder = 'data/checkpoints/'

date_start_train ='20180709140000'#'20180902000000'
date_end_train ='20190718230000'

date_start_val = '20190719170000'
date_end_val='20191003000000'


path_data='data/'


#load tarining and validation data
X_train = np.load(path_data+'train/X_three_levels_resize_from_'+date_start_train+'_to_'+date_end_train+'.npy')#s#X_total_resize_from_20180709140000_to_20190718230000.npy')#X_random_128_NO_128_YES_from_20180709140000_to_20190902070000.npy,X_random_64_NO_64_YES_from_20180709140000_to_20181029090000.npy
X_train = X_train[0:7128,:,:,:,:]
y_train = np.load(path_data+'train/Y_three_levels_resize_from_'+date_start_train+'_to_'+date_end_train+'.npy') #Y_random_128_NO_128_YES_from_20180709140000_to_20190902070000.npy,Y_random_64_NO_64_YES_from_20180709140000_to_20181029090000.npy
y_train = y_train[0:7128]#y_train[0:7128]#

X_val = np.load(path_data+'val/X_three_levels_resize_from_'+date_start_val+'_to_'+date_end_val+'.npy')#X_total_resize_from_20190719170000_to_20191003000000.npy')#X_random_49_NO_47_YES_from_20190902080000_to_20191122110000.npy,X_random_32_NO_32_YES_from_20181029100000_to_20190701150000.npy
X_val = X_val[0:1296,:,:,:,:]
y_val = np.load(path_data+'val/Y_three_levels_resize_from_'+date_start_val+'_to_'+date_end_val+'.npy')
y_val = y_val[0:1296]#y_val[0:1296]#


batch_size = 72 
nb_epoch = 2 
#------------------------------------------------------------------------------------------------------
# Load training and validation data
# Define the filenames of the training set: the training matrix X_train and the label vector y_train
file_X_train = ''
file_y_train = ''
# Define the filenames of the validation set: the validation matrix X_val and the label vector y_val
file_X_val = ''
file_y_val = ''


X_train = np.load(file_X_train)
y_train = np.load(file_y_train) 


X_val = np.load(file_X_val)
y_val = np.load(file_y_val)

# list of epochs: they have to be saved in folder
folder = 'data/checkpoints/'
list_epochs = sorted(glob.glob(folder+'lrcn_train_3levels_weighted_cat_cross_'+str(nb_epoch)+'*.hdf5'))

file_name = 'train_3levels_weighted_cat_cross_100epoch'


#----------------------------------------------------------------------------------------------------

tss_train_dict_opt_tss=[]
tss_val_dict_opt_tss=[]
hss_train_dict_opt_tss=[]
hss_val_dict_opt_tss=[]
csi_train_dict_opt_tss=[]
csi_val_dict_opt_tss=[]



####
threshold_opt_tss_weight=[]
threshold_opt_tss=[]



wtss_train_dict_opt_tss_weight=[]
wtss_val_dict_opt_tss_weight=[]
whss_train_dict_opt_tss_weight=[]
whss_val_dict_opt_tss_weight=[]
wcsi_train_dict_opt_tss_weight=[]
wcsi_val_dict_opt_tss_weight=[]

pred_train_all=np.zeros((len(list_epochs),len(y_train)))
pred_val_all=np.zeros((len(list_epochs),len(y_val)))


j=0

for file in list_epochs:
    print(file)
    model = load_model(file,compile=False)
    pred_train = model.predict(X_train)
    gc.collect()
    pred_val = model.predict(X_val)
    gc.collect()

    pred_prob=pred_train[:,1]#<-- if use cat ce
    pred_prob_val = pred_val[:,1]#<-- if use cat ce
    
    pred_train_all[j,:]=pred_prob.reshape(1,len(pred_prob))[0]
    pred_val_all[j,:]=pred_prob_val.reshape(1,len(pred_prob_val))[0]
    
    #computation of the best threshold by optimizing value-weighted skill scores (as wNSS, wTSS, wHSS, ..)
    threshold_nss_weight, metrics_training_weight, nss_vector_weight, threshold_tss_weight, threshold_hss_weight, threshold_csi_weight, threshold_tss_hss_weight = utilities.optimize_threshold_skill_scores_weight_matrix(pred_prob, y_train)
    
    
    threshold_opt_tss_weight.append(threshold_tss_weight)
    
    #computation of the best threshold by optimizing skill scores (as NSS, TSS, HSS, ..)
    threshold_nss, metrics_training, nss_vector, threshold_tss, threshold_hss, threshold_csi, threshold_tss_hss = utilities.optimize_threshold_skill_scores(pred_prob, y_train)
    
    threshold_opt_tss.append(threshold_tss)

    
    j = j + 1

    

    #compute value-weighted skill scores with optimum threshold computed by optimizing the wTSS
    #on training set
    cm, wtss, whss, wcsi  = utilities.compute_weight_cm_tss_threshold(y_train, pred_prob,threshold_tss_weight)
    wtss_train_dict_opt_tss_weight.append(wtss)
    whss_train_dict_opt_tss_weight.append(whss)
    wcsi_train_dict_opt_tss_weight.append(wcsi)
    #on validation set
    wcm_val, wtss_val, whss_val, wcsi_val = utilities.compute_weight_cm_tss_threshold(y_val, pred_prob_val,threshold_tss_weight)
    wtss_val_dict_opt_tss_weight.append(wtss_val)
    whss_val_dict_opt_tss_weight.append(whss_val)
    wcsi_val_dict_opt_tss_weight.append(wcsi_val)
    print ('threshold best wtss                 \t', threshold_tss_weight)
    
    #compute skill scores with optimum threshold computed by optimizing the TSS
    #on training set
    cm, tss, hss, csi  = utilities.compute_cm_tss_threshold(y_train, pred_prob,threshold_tss)
    tss_train_dict_opt_tss.append(tss)
    hss_train_dict_opt_tss.append(hss)
    csi_train_dict_opt_tss.append(csi)
    #on validation set
    cm_val, tss_val, hss_val, csi_val = utilities.compute_cm_tss_threshold(y_val, pred_prob_val,threshold_tss)
    tss_val_dict_opt_tss.append(tss_val)
    hss_val_dict_opt_tss.append(hss_val)
    csi_val_dict_opt_tss.append(csi_val)
    print ('threshold best tss                 \t', threshold_tss)
    
   


### (3) Computation of the level which defines the goodness of predictions on the validation set 

In [ ]:
#Function for the computation of the level which defines the goodness of predictions on the validation set

def choice_level(list_epochs,score_val_epochs,threshold_opt_score,X_val,y_val,pred_val_all,score='wtss'):
    # function which returns the optimal level which defines the goodness of predictions on the validation set
    # - X_test = the input matrix of size (number of samples, seq_length, image_shape[0], image_shape[1], number_channels))
    # - list_epochs = the file names in which the weights of the trained deep neural network are saved for all epochs
    # - score_val_epochs = vector which contains the score computed on the validation set along epochs
    # - gamma_best = a value in (0,1] which represents the optimal level computed at the third step
    # - threshold_opt = vector which contains the optimal classification thresholds along epochs    
    
    n_samples=20
    alpha_vector = numpy.zeros(n_samples)
    score_val_vector = []
    idx_list = []
    a = 0.99
    b = 0.8
    step = 1. / n_samples
    for alpha_idx in range(0, n_samples):
        alpha_level = step * alpha_idx * numpy.abs(a - b) + b
        alpha_vector[alpha_idx] = alpha_level
        pred_0_1_list=[]


        tt = np.max(np.array(score_val_epochs))*alpha_level 


        idx_ensemble=np.where(np.array(score_val_epochs)>tt)
        idx_ensemble=idx_ensemble[0]
        idx_list.append(idx_ensemblet)


        for i in idx_ensemble:#50
            file=list_epochs[i]
            print(file)
          
            pred_prob_val = pred_val_all[i,:]
            gc.collect()

            pred_0_1 = pred_prob_val > threshold_opt_score[i]
            pred_0_1_list.append(pred_0_1)

       
        pred_0_1_arr=numpy.array(pred_0_1_list)*1

        pred_median_pred_0_1=numpy.median(pred_0_1_arr,axis=0)

        idx_to_discard=numpy.where(pred_median_pred_0_1==0.5)[0]

        pred_median_pred_0_1[idx_to_discard]=1
        
        if score == 'wtss':
            wcm_val, wtss_val, whss_val, wcsi_val = utilities.compute_weight_cm_tss(y_val, pred_median_pred_0_1)
            print('alpha_level = ',alpha_level) 
            print('Weighted Skill scores (wtss optimization)')
            print(wcm_val)
            print('wtss = ','{:0.4f}'.format(wtss_val))
            print('whss = ','{:0.4f}'.format(whss_val))
            print('wcsi = ','{:0.4f}'.format(wcsi_val))
            score_val_vector.append(wtss_val)
            
        if score == 'tss':
            cm_val, tss_val, hss_val, csi_val = utilities.compute_cm_tss(y_val, pred_median_pred_0_1)#_weight
            print('alpha_level = ',alpha_level) 
            print('Skill scores (tss optimization)')
            print(cm_val)
            print('tss = ','{:0.4f}'.format(tss_val))
            print('hss = ','{:0.4f}'.format(hss_val))
            print('csi = ','{:0.4f}'.format(csi_val))
            score_val_vector.append(tss_val)
        
    score_val_vector = np.array(score_val_vector)
    idx_alpha_best = np.where(score_val_vector == np.max(score_val_vector))
    alpha_best = alpha_vector[idx_alpha_best]
    idx_list = np.array(idx_list)
    idx_best = idx_list[idx_alpha_best]
    return alpha_best, score_val_vector, alpha_vector, idx_alpha_best, idx_best
        


In [ ]:
# Compute the level  which defines the goodness of predictions on the validation set (the goodness is measured 
# w.r.t. the desired skill score)

# the level alpha in the case of the wtss optimization strategy 
gamma_best_weight, wtss_val_vector_thresholds, alpha_vector_weight,idx_gamma_best_weight,idx_weight_best=choice_level(list_epochs,
                                                                                  wtss_val_dict_opt_tss_weight,
                                                                                  threshold_opt_tss_weight,
                                                                                  X_val,y_val,pred_val_all)
print('gamma computed with the wtss optimization strategy: ', gamma_best_weight)



# the level alpha in the case of the tss optimization strategy 
gamma_best, tss_val_vector_thresholds, alpha_vector,idx_gamma_best,idx_best=choice_level(list_epochs,
                                                                                  tss_val_dict_opt_tss,
                                                                                  threshold_opt_tss,
                                                                                  X_val,y_val,pred_val_all)

print('gamma computed with the tss optimization strategy: ', gamma_best)



#save variables in data/results
#np.save('data/results/wtss_val_vector_thresholds_'+file_name+'.npy',wtss_val_vector_thresholds)
#np.save('data/results/gamma_wtss_'+file_name+'.npy',gamma_best_weight)
#np.save('data/results/tss_val_vector_thresholds_'+file_name+'.npy',tss_val_vector_thresholds)
#np.save('data/results/gamma_tss_'+file_name+'.npy',gamma_best)


### (4) Prediction on the test set

In [ ]:
#Function which defines predictions

def ensemble_prediction(X_test,list_epochs,score_val_epochs,gamma_best,threshold_opt):#wtss_val_dict_opt_tss_weig
    # function which returns the binary prediction using the ensemble method given in input X_test
    # - X_test = the input matrix of size (number of samples, seq_length, image_shape[0], image_shape[1], number_channels))
    # - list_epochs = the file names in which the weights of the trained deep neural network are saved for all epochs
    # - score_val_epochs = vector which contains the score computed on the validation set along epochs
    # - gamma_best = a value in (0,1] which represents the optimal level computed at the third step
    # - threshold_opt = vector which contains the optimal classification thresholds along epochs
    
    pred_0_1_list=[]


    alpha = np.max(np.array(score_val_epochs))*gamma_best[0] 


    idx_epochs=np.where(np.array(score_val_epochs)>alpha)
    idx_epochs=idx_epochs[0]
    print('List of epochs involved in the ensemble prediction')
    for i in idx_epochs:
        file=list_epochs[i]
        print(file)
        model = load_model(file,compile=False)
        pred_test = model.predict(X_test)
        pred_prob_test =pred_test[:,1]#if use cat CE..
        pred_prob_test = pred_prob_test.reshape(1,len(pred_prob_test))
        pred_prob_test = pred_prob_test[0]
        gc.collect()

        pred_0_1 = pred_prob_test > threshold_opt[i]
        pred_0_1_list.append(pred_0_1)

    pred_0_1_arr=numpy.array(pred_0_1_list)*1

    pred_test=numpy.median(pred_0_1_arr,axis=0)

    idx_to_discard=numpy.where(pred_test==0.5)[0]
    pred_test[idx_to_discard]=1
    
    return pred_test

In [ ]:
# Predict on a test set

folder = 'data/checkpoints/'

# define the file names of test data: the test matrix X_test and the test label vector y_test
file_X_test = ''
file_y_test = ''


# The matrix X_test must be of size (number of test samples, seq_length, image_shape[0], image_shape[1], number_channels))
X_test = np.load(file_X_test) 
y_test = np.load(file_y_test)

print('**** wTSS optimization strategy ****')
pred_test_wtss_strategy = ensemble_prediction(X_test,list_epochs,wtss_val_dict_opt_tss_weight,gamma_best_weight,threshold_opt_tss_weight)
print('**** TSS optimization strategy ****')
pred_test_tss_strategy = ensemble_prediction(X_test,list_epochs,tss_val_dict_opt_tss,gamma_best,threshold_opt_tss)

In [ ]:
# Results on the test set

##################################wTSS optimization strategy############################################

#compute scores on test set
cm_test, tss_test, hss_test, csi_test = utilities.compute_cm_tss(y_test, pred_test_wtss_strategy)
print('**** wTSS optimization strategy ****')
print('Skill scores on the test set')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

#compute value-weighted skill scores on test set
wcm_test, wtss_test, whss_test, wcsi_test = utilities.compute_weight_cm_tss(y_test, pred_test_wtss_strategy)
print('Value-weighted Skill scores on the test set')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

###############################TSS optimization strategy##############################################
print('**** TSS optimization strategy ****')
#compute scores on test set
cm_test, tss_test, hss_test, csi_test = utilities.compute_cm_tss(y_test, pred_test_tss_strategy)
print('Skill scores on the test set')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

#compute value-weighted skill scores on test set
wcm_test, wtss_test, whss_test, wcsi_test = utilities.compute_weight_cm_tss(y_test, pred_test_tss_strategy)
print('Value-weighted skill scores on the test set')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

